In [1]:
import tkinter as tk
import matplotlib.pyplot as plt
import matplotlib.widgets as wgt
import scipy.signal as sc
import pandas as pd
from tkinter import *
from tkinter import ttk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from scipy.signal import iirfilter, filtfilt

In [ ]:

def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, cutoff_frequency, current_noise=None):
  #print(phase)

  w = 2* np.pi * frequency
  t = np.linspace(0, 10, 1000)
  y_clean = amplitude * np.sin(w * t + phase)
  if current_noise is None:
    noise = noise_mean + np.sqrt(noise_covariance) * np.random.normal(size=t.shape) 
  else: 
     noise = current_noise
  y_noisy = y_clean + noise
  # noise = np.random.normal(loc=noise_mean, scale=np.sqrt(noise_covariance), size=n)
  nyquist = 0.5 * (1 / (t[1] - t[0]))  
  normalized_cutoff = cutoff_frequency / nyquist
  b, a = iirfilter(N=4, Wn=normalized_cutoff, btype='low', ftype='butter')

  y_filtered = filtfilt(b, a, y_noisy)

  return t, y_clean, y_noisy, y_filtered, noise


  #return t, y_clean, y_noisy

class SuperShapeFrame(Frame):

  def __init__(self, master = None):

    Frame.__init__(self, master)
    self.grid()

    self.m = 0.97
    self.n1 = 0.267
    self.n2 = 0
    self.n3 = 0.108
    self.n4 = 0.101
    self.n5 = 5



    self.fig = plt.Figure(figsize=(8, 6), dpi=80)
    self.ax = self.fig.add_subplot(111)
    self.ax.set_title("Графік гармоніки")
    self.ax.grid(True)
    canvas = FigureCanvasTkAgg(self.fig, master=self)
    canvas.get_tk_widget().grid(row=0, column=0, columnspan=4)

    self.show_noise = BooleanVar(value=True) 
    self.show_filtered = BooleanVar(value=False)
    
    
    label = Label(self, text = 'Amplitude')
    label.grid(row = 1, column = 1)
    self.amplitude_slider = Scale(self, from_ = 0, to = 2, resolution=0.001, length=200, orient = HORIZONTAL, command = lambda i : self.update_m())

    self.amplitude_slider.grid(row = 1, column = 2)

    label = Label(self, text = 'Frequency')
    label.grid(row = 2, column = 1)
    self.n1_slider = Scale(self, from_ = 0, to = 1, resolution=0.001, length=200, orient = HORIZONTAL, command = lambda i : self.update_n1())
    self.n1_slider.grid(row = 2, column = 2)

    label = Label(self, text = 'Phase')
    label.grid(row = 3, column = 1)
    self.n2_slider = Scale(self, from_ = 0, to = 1, resolution=0.001, length=200, orient = HORIZONTAL, command = lambda i : self.update_n2())
    self.n2_slider.grid(row = 3, column = 2)

    label = Label(self, text = 'Noise Mean')
    label.grid(row = 4, column = 1)
    self.n3_slider = Scale(self, from_ = 0, to = 0.2, resolution=0.001,length=200, orient = HORIZONTAL, command = lambda i : self.update_n3())
    self.n3_slider.grid(row = 4, column = 2)

    label = Label(self, text = 'Noise Covariance')
    label.grid(row = 5, column = 1)
    self.n4_slider = Scale(self, from_ = 0, to = 1, resolution=0.001, length=200, orient = HORIZONTAL, command = lambda i : self.update_n4())

    self.n4_slider.grid(row = 5, column = 2)

    self.noise_checkbox = Checkbutton(self, text="Показник шуму", variable=self.show_noise, command=self.refresh_figure)
    self.noise_checkbox.grid(row=7, column=1, columnspan=2)

    self.noise_checkbox = Checkbutton(self, text="Відфільрована", variable=self.show_filtered, command=self.refresh_figure)
    self.noise_checkbox.grid(row=7, column=2, columnspan=2)

    
    label = Label(self, text = 'Reset')
    label.grid(row = 8, column = 1) 
    self.button = Button(self, text = "Reset", command = self.f, bg="red", activebackground="darkred")
    self.button.grid(row = 8, column = 2)





    label = Label(self, text = 'Cutoff Frequency')
    label.grid(row = 6, column = 1)
    self.n5_slider = Scale(self, from_ = 0, to = 10, resolution=0.001, length=200, orient = HORIZONTAL, command = lambda i : self.update_n5())
    self.n5_slider.grid(row = 6, column = 2)

    self.draw_initial_figure()

    
  def f(self):
        #print("Reset")
      
        self.button['bg'] = 'green' 
        self.button['activebackground'] = 'darkgreen'
        self.button.update()
        self.m = 0.97
        self.n1 = 0.267
        self.n2 = 0
        self.n3 = 0.108
        self.n4 = 0.101
        self.n5 = 5

        self.amplitude_slider.set(self.m)
        self.n1_slider.set(self.n1)
        self.n2_slider.set(self.n2)
        self.n3_slider.set(self.n3)
        self.n4_slider.set(self.n4)
        self.n5_slider.set(self.n5)
        self.refresh_figure()


  def update_m(self):
    self.m = self.amplitude_slider.get()
    self.refresh_figure()

  def update_n1(self):
    print(self.n1_slider.get())
    self.n1 = self.n1_slider.get()
    print(self.n1)
    self.refresh_figure()

  def update_n2(self):
    self.n2 = (self.n2_slider.get())
    self.refresh_figure()

  def update_n3(self):
    self.n3 = self.n3_slider.get()
    self.current_noise = None
    self.refresh_figure()

  def update_n4(self):
    self.n4 = self.n4_slider.get()
    self.current_noise = None
    self.refresh_figure()

  def update_n5(self):
    self.n5 = self.n5_slider.get()
    self.refresh_figure()

  # photo = 
  def refresh_figure(self):
        t, y_clean, y_noisy, y_f, noise = harmonic_with_noise(self.m, self.n1, self.n2, self.n3, self.n4, self.n5, self.current_noise)
        self.ax.cla()
        #self.lines_clean.set_ydata(y_clean)
        
        self.current_noise = noise

        if self.show_filtered.get():
          self.lines_clean, = self.ax.plot(t, y_f, "b-", label="Filtered data")
          self.ax.plot(t, y_clean, "g--", label="Чиста гармоніка")
        else:
          self.lines_clean, = self.ax.plot(t, y_clean, "b-", label="ЧЧиста гармоніка")

        if self.show_noise.get():
          self.lines_noisy, = self.ax.plot(t, y_noisy, "orange", alpha=0.6, label="Шум")
        else:
          self.lines_noisy, = self.ax.plot([], [], "orange", alpha=0.6, label="Шум")  

        self.fig.canvas.draw_idle()

  def draw_initial_figure(self):
        self.current_noise = None
        t, y_clean, y_noisy, y_filtered, noise = harmonic_with_noise(self.m, self.n1, self.n2, self.n3, self.n4, self.n5, self.current_noise)
        self.current_noise = noise
        self.lines_clean, = self.ax.plot(t, y_clean, "b-", label="Чиста гармоніка")
        self.lines_noisy, = self.ax.plot(t, y_noisy, "orange", alpha=0.6, label="Шум")
        if self.show_filtered.get():
          self.ax.plot(t, y_filtered, "g-", label="Відфільтрована")
        self.ax.legend()
        self.fig.canvas.draw()
    
app = SuperShapeFrame()
app.master.title('harmonic function')
app.mainloop()


0.001
0.001
0.002
0.002
0.003
0.003
0.012
0.012
0.089
0.089
0.125
0.125
0.131
0.131
0.143
0.143
0.155
0.155
0.167
0.167
0.185
0.185
0.202
0.202
0.22
0.22
0.232
0.232
0.238
0.238
0.262
0.262
0.333
0.333
0.369
0.369
0.381
0.381
0.405
0.405
0.435
0.435
0.44
0.44
0.446
0.446
0.452
0.452
0.458
0.458
0.464
0.464
0.476
0.476
0.5
0.5
0.536
0.536
0.571
0.571
0.607
0.607
0.631
0.631
0.637
0.637
0.661
0.661
0.667
0.667
0.69
0.69
0.696
0.696
0.714
0.714
0.732
0.732
0.738
0.738
0.744
0.744
0.75
0.75
0.756
0.756
0.762
0.762
0.768
0.768
0.774
0.774
0.78
0.78
0.786
0.786
0.792
0.792
0.798
0.798
0.804
0.804
0.81
0.81
0.815
0.815
0.827
0.827
0.833
0.833
0.839
0.839
0.845
0.845
0.851
0.851
0.857
0.857
0.863
0.863
0.869
0.869
0.875
0.875
0.881
0.881
0.887
0.887
0.893
0.893
0.905
0.905
0.911
0.911
0.917
0.917
0.923
0.923
0.929
0.929
0.935
0.935
0.94
0.94
0.946
0.946
0.952
0.952
0.958
0.958
0.964
0.964
0.97
0.97
0.976
0.976
0.982
0.982
0.988
0.988
0.994
0.994
1.0
1.0
0.994
0.994
0.964
0.964
0.946
0.946
0.92

: 

In [3]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output, State

# Функція для генерації гармоніки із шумом
def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise, time):
    harmonic = amplitude * np.sin(2 * np.pi * frequency * time + phase)
    noise = np.random.normal(noise_mean, np.sqrt(noise_covariance), size=len(time))
    if show_noise:
        return harmonic + noise, noise
    return harmonic, None

# Функція для кастомного фільтру
def custom_filter(signal, window_size):
    if window_size < 1:
        raise ValueError("Розмір вікна має бути >= 1")

    kernel = np.ones(window_size) / window_size
    filtered_signal = np.convolve(signal, kernel, mode='same')
    return filtered_signal

# Початкові параметри
initial_params = {
    "amplitude": 1.0,
    "frequency": 1.0,
    "phase": 0.0,
    "noise_mean": 0.0,
    "noise_covariance": 0.1,
    "show_noise": True,
    "filter_window_size": 5
}

# Діапазон часу
time = np.linspace(0, 2, 1000)

# Створення Dash-додатку
app = Dash(__name__)

# Інтерфейс користувача
app.layout = html.Div([
    html.H1("Гармоніка із шумом"),

    dcc.Graph(id="harmonic-plot"),

    html.Div([
        html.Div([
            html.Label(id="amplitude-label", children=f"Амплітуда: {initial_params['amplitude']:.1f}"),
            dcc.Slider(id="amplitude-slider", min=0.1, max=5.0, step=0.1, value=initial_params["amplitude"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="frequency-label", children=f"Частота: {initial_params['frequency']:.1f}"),
            dcc.Slider(id="frequency-slider", min=0.1, max=5.0, step=0.1, value=initial_params["frequency"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="phase-label", children=f"Фазовий зсув: {initial_params['phase']:.1f}"),
            dcc.Slider(id="phase-slider", min=0.0, max=2 * np.pi, step=0.1, value=initial_params["phase"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-mean-label", children=f"Середнє шуму: {initial_params['noise_mean']:.1f}"),
            dcc.Slider(id="noise-mean-slider", min=-1.0, max=1.0, step=0.1, value=initial_params["noise_mean"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-covariance-label", children=f"Дисперсія шуму: {initial_params['noise_covariance']:.2f}"),
            dcc.Slider(id="noise-covariance-slider", min=0.01, max=1.0, step=0.01, value=initial_params["noise_covariance"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="filter-window-label", children=f"Розмір вікна фільтру: {initial_params['filter_window_size']}"),
            dcc.Slider(id="filter-window-slider", min=1, max=50, step=1, value=initial_params["filter_window_size"]),
        ], style={"margin-bottom": "20px"}),

        html.Label("Відображення шуму"),
        dcc.Checklist(id="show-noise-checkbox", options=[{"label": "Шум", "value": "show_noise"}],
                      value=["show_noise"] if initial_params["show_noise"] else []),

        html.Label("Вибір графіка"),
        dcc.Dropdown(
            id="plot-dropdown",
            options=[
                {"label": "Тільки гармоніка", "value": "harmonic"},
                {"label": "Тільки шум", "value": "noise"},
                {"label": "Обидва", "value": "both"},
                {"label": "Фільтрована гармоніка", "value": "filtered"},
            ],
            value="both",
        ),

        html.Button("Reset", id="reset-button", n_clicks=0),
    ], style={"width": "50%", "margin": "auto"})
])

# Оновлення графіка
@app.callback(
    Output("harmonic-plot", "figure"),
    Input("amplitude-slider", "value"),
    Input("frequency-slider", "value"),
    Input("phase-slider", "value"),
    Input("noise-mean-slider", "value"),
    Input("noise-covariance-slider", "value"),
    Input("filter-window-slider", "value"),
    Input("show-noise-checkbox", "value"),
    Input("plot-dropdown", "value"),
    State("harmonic-plot", "figure")
)
def update_plot(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window, show_noise, plot_selection, current_figure):
    show_noise_flag = "show_noise" in show_noise
    harmonic, noise = harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise_flag, time)

    if plot_selection == "filtered":
        harmonic = custom_filter(harmonic, filter_window)

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Гармоніка", "Шум"))

    # Відображення графіків залежно від вибору у drop-down меню
    if plot_selection in ["harmonic", "both", "filtered"]:
        fig.add_trace(go.Scatter(x=time, y=harmonic, mode="lines", name="Гармоніка"), row=1, col=1)

    if plot_selection in ["noise", "both"] and noise is not None:
        fig.add_trace(go.Scatter(x=time, y=noise, mode="lines", name="Шум", line=dict(dash="dot")), row=2, col=1)

    fig.update_layout(height=600, title="Гармоніка із шумом")

    return fig

# Оновлення підписів слайдерів
@app.callback(
    [Output("amplitude-label", "children"),
     Output("frequency-label", "children"),
     Output("phase-label", "children"),
     Output("noise-mean-label", "children"),
     Output("noise-covariance-label", "children"),
     Output("filter-window-label", "children")],
    [Input("amplitude-slider", "value"),
     Input("frequency-slider", "value"),
     Input("phase-slider", "value"),
     Input("noise-mean-slider", "value"),
     Input("noise-covariance-slider", "value"),
     Input("filter-window-slider", "value")]
)
def update_labels(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window):
    return (
        f"Амплітуда: {amplitude:.1f}",
        f"Частота: {frequency:.1f}",
        f"Фазовий зсув: {phase:.1f}",
        f"Середнє шуму: {noise_mean:.1f}",
        f"Дисперсія шуму: {noise_covariance:.2f}",
        f"Розмір вікна фільтру: {filter_window}"
    )

# Скидання параметрів
@app.callback(
    [Output("amplitude-slider", "value"),
     Output("frequency-slider", "value"),
     Output("phase-slider", "value"),
     Output("noise-mean-slider", "value"),
     Output("noise-covariance-slider", "value"),
     Output("filter-window-slider", "value"),
     Output("show-noise-checkbox", "value"),
     Output("plot-dropdown", "value")],
    Input("reset-button", "n_clicks")
)
def reset_parameters(n_clicks):
    return (
        initial_params["amplitude"],
        initial_params["frequency"],
        initial_params["phase"],
        initial_params["noise_mean"],
        initial_params["noise_covariance"],
        initial_params["filter_window_size"],
        ["show_noise"] if initial_params["show_noise"] else [],
        "both",
    )

# Запуск додатку
app.run_server(debug=True)
